In [1]:
#cnn with xgboost
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Input, Dropout
from keras.optimizers import Adam

# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data('mnist.npz')
print(y_train[:1])
# data pre-processing
X_train = X_train.reshape(-1, 1,28, 28)/255.
X_test = X_test.reshape(-1, 1,28, 28)/255.
Y_train = np_utils.to_categorical(y_train, num_classes=10)
Y_test = np_utils.to_categorical(y_test, num_classes=10)
print(Y_train[:1])

# Another way to build your CNN
inputs = Input(shape=(1,28,28))

conv2d1 = Convolution2D(filters=64, kernel_size=5, padding='same', data_format='channels_first')(inputs)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling2D(pool_size=2, strides=2, padding='same', data_format='channels_first')(activation1)
dropout1 = Dropout(0.25)(maxpooling1)

conv2d2 = Convolution2D(filters=64, kernel_size=4, padding='same', data_format='channels_first')(dropout1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling2D(pool_size=2, strides=2, padding='same', data_format='channels_first')(activation2)
# dropout2 = Dropout(0.25)(maxpooling2)

flatten = Flatten()(maxpooling2)

dense1 = Dense(1024)(flatten)
activation3 = Activation('relu')(dense1)

dense2 = Dense(10)(activation3)
activation4 = Activation('softmax')(dense2)

model_dense2_output = Model(inputs=inputs, outputs=activation3)
model = Model(inputs=inputs, outputs=activation4)


adam = Adam(lr=1e-4)

# We add metrics to get more results you want to see
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

print('Training ------------')
# Another way to train the model
model.fit(X_train, Y_train, epochs=5, batch_size=16)

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(X_test, Y_test)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

#with xgboost
import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.grid_search import GridSearchCV

X_train_xg = model_dense2_output.predict(X_train)
X_test_xg = model_dense2_output.predict(X_test)
# print(np.array(X_train_xg).shape)
# print(np.array(y_train).shape)

z = np.concatenate([np.array(X_train_xg).reshape(60000,1024),np.array(y_train).reshape(60000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('train_xg.csv',index=False)

z = np.concatenate([np.array(X_test_xg).reshape(10000,1024),np.array(y_test).reshape(10000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('test_xg.csv',index=False)

train = pd.read_csv('train_xg.csv')
train_y = train['1024'].astype('int')
train_x = train.drop(['1024'],axis=1)
dataset = xgboost.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class':10,'objective':'multi:softmax' } 
model_xg = xgboost.train(params, dataset, num_boost_round=150, evals=watchlist)

test = pd.read_csv('test_xg.csv')
test_y = test['1024'].astype('int')
test_x = test.drop(['1024'],axis=1)

test_x = xgboost.DMatrix(test_x)
result = model_xg.predict(test_x)
#print(t)
#m = LogisticRegression()
#m = SVC(kernel='rbf',decision_function_shape='ovr')
#m.fit(X,y)
#print(m)

print(metrics.classification_report(test_y, result))
print(metrics.confusion_matrix(test_y, result))

Using TensorFlow backend.


[5]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]]
Training ------------
Epoch 1/5
60000/60000 [==============================] - 799s 13ms/step - loss: 0.1814 - acc: 0.9459
Epoch 2/5
60000/60000 [==============================] - 797s 13ms/step - loss: 0.0582 - acc: 0.9821
Epoch 3/5
60000/60000 [==============================] - 796s 13ms/step - loss: 0.0386 - acc: 0.9881
Epoch 4/5
60000/60000 [==============================] - 798s 13ms/step - loss: 0.0283 - acc: 0.9913
Epoch 5/5
60000/60000 [==============================] - 796s 13ms/step - loss: 0.0214 - acc: 0.9926

Testing ------------
10000/10000 [==============================] - 56s 6ms/step
('\ntest loss: ', 0.027207671511101945)
('\ntest accuracy: ', 0.99129999999999996)


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[0]	train-merror:0.013917
[1]	train-merror:0.009367
[2]	train-merror:0.008083
[3]	train-merror:0.006783
[4]	train-merror:0.0062
[5]	train-merror:0.005683
[6]	train-merror:0.005283
[7]	train-merror:0.004917
[8]	train-merror:0.004583
[9]	train-merror:0.0044
[10]	train-merror:0.00405
[11]	train-merror:0.003733
[12]	train-merror:0.003317
[13]	train-merror:0.003117
[14]	train-merror:0.00295
[15]	train-merror:0.0028
[16]	train-merror:0.00265
[17]	train-merror:0.00255
[18]	train-merror:0.002333
[19]	train-merror:0.002217
[20]	train-merror:0.002067
[21]	train-merror:0.001867
[22]	train-merror:0.001767
[23]	train-merror:0.0017
[24]	train-merror:0.00155
[25]	train-merror:0.001433
[26]	train-merror:0.001217
[27]	train-merror:0.001083
[28]	train-merror:0.0009
[29]	train-merror:0.0008
[30]	train-merror:0.0007
[31]	train-merror:0.000633
[32]	train-merror:0.000583
[33]	train-merror:0.0005
[34]	train-merror:0.000433
[35]	train-merror:0.0004
[36]	train-merror:0.000383
[37]	train-merror:0.00035
[38]	tra

In [3]:
#use rnn and xgboost
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import Adam
import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.grid_search import GridSearchCV

TIME_STEPS = 28     # same as the height of the image
INPUT_SIZE = 28     # same as the width of the image
BATCH_SIZE = 50
BATCH_INDEX = 0
OUTPUT_SIZE = 10
CELL_SIZE = 50
LR = 0.001


# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data('mnist.npz')

# data pre-processing
X_train = X_train.reshape(-1, 28, 28) / 255.      # normalize
X_test = X_test.reshape(-1, 28, 28) / 255.        # normalize
Y_train = np_utils.to_categorical(y_train, num_classes=10)
Y_test = np_utils.to_categorical(y_test, num_classes=10)

inputs = Input(shape = (28,28))
#srnn = SimpleRNN(units=CELL_SIZE,input_dim=INPUT_SIZE, input_length=TIME_STEPS )(inputs)
srnn = LSTM(units=CELL_SIZE,input_dim=INPUT_SIZE, input_length=TIME_STEPS )(inputs)
dense = Dense(OUTPUT_SIZE,activation='softmax')(srnn)

model_dense2_output = Model(inputs=inputs, outputs=srnn)
model = Model(inputs=inputs, outputs=dense)

# optimizer
adam = Adam(LR)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
# training
print('Training ------------')
model.fit(X_train, Y_train, epochs=5, batch_size=16)

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
result = model.predict(X_test)
loss, accuracy = model.evaluate(X_test, Y_test)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

a = np.argmax(Y_test,axis = 1)
b = np.argmax(result,axis = 1)
print(metrics.classification_report(a, b))
print(metrics.confusion_matrix(a, b))

#with xgboost
X_train_xg = model_dense2_output.predict(X_train)
X_test_xg = model_dense2_output.predict(X_test)

z = np.concatenate([np.array(X_train_xg).reshape(60000,50),np.array(y_train).reshape(60000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('train_xg_rnn.csv',index=False)

z = np.concatenate([np.array(X_test_xg).reshape(10000,50),np.array(y_test).reshape(10000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('test_xg_rnn.csv',index=False)

train = pd.read_csv('train_xg_rnn.csv')
train_y = train['50'].astype('int')
train_x = train.drop(['50'],axis=1)
dataset = xgboost.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class':10,'objective':'multi:softmax' } 
model_xg = xgboost.train(params, dataset, num_boost_round=100, evals=watchlist)

test = pd.read_csv('test_xg_rnn.csv')
test_y = test['50'].astype('int')
test_x = test.drop(['50'],axis=1)

test_x = xgboost.DMatrix(test_x)
result = model_xg.predict(test_x)
#print(t)
#m = LogisticRegression()
#m = SVC(kernel='rbf',decision_function_shape='ovr')
#m.fit(X,y)
#print(m)

print(metrics.classification_report(test_y, result))
print(metrics.confusion_matrix(test_y, result))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:38: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, input_shape=(28, 28))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                15800     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
Total params: 16,310
Trainable params: 16,310
Non-trainable params: 0
_________________________________________________________________
None
Training ------------
Epoch 1/5
60000/60000 [==============================] - 71s 1ms/step - loss: 0.3741 - acc: 0.8834
Epoch 2/5
60000/60000 [==============================] - 70s 1ms/step - loss: 0.1343 - acc: 0.9595
Epoch 3/5
60000/60000 [==============================] - 70s 1ms/step - loss: 0.0922 - acc: 0.9718
Epoch 4/5
60000/60000 [==============================] - 70s 1ms/step - loss: 

In [4]:
#only xgboost
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.grid_search import GridSearchCV

# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data('mnist.npz')

# data pre-processing
X_train = X_train.reshape(-1, 28*28) / 255.      # normalize
X_test = X_test.reshape(-1, 28*28) / 255.        # normalize
X_train.shape

z = np.concatenate([np.array(X_train).reshape(60000,784),np.array(y_train).reshape(60000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('train_xg_only.csv',index=False)

z = np.concatenate([np.array(X_test).reshape(10000,784),np.array(y_test).reshape(10000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('test_xg_only.csv',index=False)

train = pd.read_csv('train_xg_only.csv')
train_y = train['784'].astype('int')
train_x = train.drop(['784'],axis=1)
dataset = xgboost.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class':10,'objective':'multi:softmax' } 
model_xg = xgboost.train(params, dataset, num_boost_round=100, evals=watchlist)

test = pd.read_csv('test_xg_only.csv')
test_y = test['784'].astype('int')
test_x = test.drop(['784'],axis=1)

test_x = xgboost.DMatrix(test_x)
result = model_xg.predict(test_x)
#print(t)
#m = LogisticRegression()
#m = SVC(kernel='rbf',decision_function_shape='ovr')
#m.fit(X,y)
#print(m)

print(metrics.classification_report(test_y, result))
print(metrics.confusion_matrix(test_y, result))

[0]	train-merror:0.103083
[1]	train-merror:0.08205
[2]	train-merror:0.071533
[3]	train-merror:0.064783
[4]	train-merror:0.060833
[5]	train-merror:0.0577
[6]	train-merror:0.054833
[7]	train-merror:0.0521
[8]	train-merror:0.049417
[9]	train-merror:0.047383
[10]	train-merror:0.0452
[11]	train-merror:0.042817
[12]	train-merror:0.04085
[13]	train-merror:0.039467
[14]	train-merror:0.038
[15]	train-merror:0.03615
[16]	train-merror:0.034917
[17]	train-merror:0.03355
[18]	train-merror:0.0324
[19]	train-merror:0.031233
[20]	train-merror:0.029817
[21]	train-merror:0.028733
[22]	train-merror:0.027383
[23]	train-merror:0.026283
[24]	train-merror:0.025183
[25]	train-merror:0.024417
[26]	train-merror:0.0235
[27]	train-merror:0.022617
[28]	train-merror:0.021817
[29]	train-merror:0.020817
[30]	train-merror:0.019783
[31]	train-merror:0.019067
[32]	train-merror:0.018283
[33]	train-merror:0.0176
[34]	train-merror:0.016983
[35]	train-merror:0.016333
[36]	train-merror:0.015617
[37]	train-merror:0.015217
[38

In [7]:
#aotoencoder with xgboost

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input

# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(x_train, y_train), (x_test, y_test) = mnist.load_data('mnist.npz')

# data pre-processing
x_train = x_train.astype('float32') / 255. - 0.5       # minmax_normalized
x_test = x_test.astype('float32') / 255. - 0.5         # minmax_normalized

x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

print(x_train.shape)
print(x_test.shape)

# in order to plot in a 2D figure
encoding_dim = 10
# this is our input placeholder
input_img = Input(shape=(784,))
# encoder layers
encoded = Dense(256, activation='relu')(input_img)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoder_output = Dense(encoding_dim)(encoded)

# decoder layers
decoded = Dense(64, activation='relu')(encoder_output)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(784, activation='tanh')(decoded)

# construct the autoencoder model
autoencoder = Model(input=input_img, output=decoded)
# construct the encoder model for plotting
encoder = Model(input=input_img, output=encoder_output)
# compile autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
# training
autoencoder.fit(x_train, x_train,epochs=5, batch_size=16,shuffle=True)

import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.grid_search import GridSearchCV

X_train_xg = encoder.predict(x_train)
X_test_xg = encoder.predict(x_test)

z = np.concatenate([np.array(X_train_xg).reshape(60000,10),np.array(y_train).reshape(60000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('train_xg_ae.csv',index=False)

z = np.concatenate([np.array(X_test_xg).reshape(10000,10),np.array(y_test).reshape(10000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('test_xg_ae.csv',index=False)

train = pd.read_csv('train_xg_ae.csv')
train_y = train['10'].astype('int')
train_x = train.drop(['10'],axis=1)
dataset = xgboost.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class':10,'objective':'multi:softmax' } 
model_xg = xgboost.train(params, dataset, num_boost_round=100, evals=watchlist)

test = pd.read_csv('test_xg_ae.csv')
test_y = test['10'].astype('int')
test_x = test.drop(['10'],axis=1)

test_x = xgboost.DMatrix(test_x)
result = model_xg.predict(test_x)
#print(t)
#m = LogisticRegression()
#m = SVC(kernel='rbf',decision_function_shape='ovr')
#m.fit(X,y)
#print(m)

print(metrics.classification_report(test_y, result))
print(metrics.confusion_matrix(test_y, result))

(60000, 784)
(10000, 784)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


Epoch 1/5
60000/60000 [==============================] - 36s 594us/step - loss: 0.0287
Epoch 2/5
60000/60000 [==============================] - 35s 585us/step - loss: 0.0215
Epoch 3/5
60000/60000 [==============================] - 35s 586us/step - loss: 0.0198
Epoch 4/5
60000/60000 [==============================] - 35s 590us/step - loss: 0.0188
Epoch 5/5
60000/60000 [==============================] - 35s 592us/step - loss: 0.0182
[0]	train-merror:0.118033
[1]	train-merror:0.101167
[2]	train-merror:0.093783
[3]	train-merror:0.088967
[4]	train-merror:0.084983
[5]	train-merror:0.081833
[6]	train-merror:0.079867
[7]	train-merror:0.077567
[8]	train-merror:0.074867
[9]	train-merror:0.07305
[10]	train-merror:0.071417
[11]	train-merror:0.06945
[12]	train-merror:0.067667
[13]	train-merror:0.066017
[14]	train-merror:0.0647
[15]	train-merror:0.062833
[16]	train-merror:0.061567
[17]	train-merror:0.06025
[18]	train-merror:0.058817
[19]	train-merror:0.057567
[20]	train-merror:0.056467
[21]	train-me